In [1]:
import random

# Function that generates another function given a list of coefficient.
# (also determining the polynomial's degree)
# https://en.wikipedia.org/wiki/Shamir%27s_Secret_Sharing#Usage
def generate_polynomial_fn(a_vars):
  parts = []
  for degree in range(len(a_vars)):
    # closure on variable 'degree' using default parameter
    def polynomial_part(x, d=degree):
      return a_vars[d] * x**d
    parts.append(polynomial_part)
    
  # Returns a lambda that sums all part of the polynomial for a given 'x'
  return lambda x: sum(map(lambda f: f(x), parts))

In [2]:
def shamir_split(m, n, secret):
  # Choosing random numbers that are not too far away from our secret
  rnds = [int(random.uniform(0.2, 0.8) * secret) for _ in range(m - 1)]
  f = generate_polynomial_fn([secret] + rnds)
  # returns the evaluation of the polynomial function for 0 < x < n+1
  # WARNING: here x=0 would reveal the secret (f(0) == secret)
  return [(x, f(x)) for x in range(1, n + 1)]

In [3]:
from z3 import *

# https://en.wikipedia.org/wiki/Shamir%27s_Secret_Sharing#Solution
def shamir_resolve(splits):
  degree = len(splits)
  
  # Z3 way of declaring unknown variables
  a = [Int(f'a{i}') for i in range(0, degree)]
  solver = Solver()
  
  # Create a simple system of equations from the splits
  for x, y in splits:
    p = generate_polynomial_fn(a)
    solver.add(y == p(x))
  
  # This part is Z3 specific, we check and solve the system of equations
  solver.check()
  model = solver.model()
  sec_int = model[a[0]].as_long()
  
  return sec_int

In [57]:
secret =10

In [59]:
m = 2 # number of parties needed to reconstruct secret
n = 2 # total number of parties
splits = shamir_split(m, n, secret)
assert(len(splits) == n)
print(splits)

[(1, 13), (2, 16)]


In [30]:
# Picking arbitrarily a subset of the splits
split_subset = splits[0:2]

# Reconstruct the secret using only the splits (and the 'x' value)
shamir_resolve(split_subset)

4

In [90]:
a=[(1,171), (2,222)]

In [91]:
shamir_resolve(a)

120